# CS 109A/AC 209A/STAT 121A Data Science: Data Driven March Madness Milestone #3
**Harvard University**<br>
**Fall 2016**<br>
**Instructors: W. Pan, P. Protopapas, K. Rader**<br>
**Due Date: ** Saturday, November 6th, 2016 at 11:59pm

In [28]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import random
from mpl_toolkits.mplot3d import Axes3D
import csv
from sklearn import linear_model
%matplotlib inline

In [4]:
#notes
#regular season is like training set, tourney results are like testing set
#should boil down the regular season set with just tourney teams


**Loading in Each Dataframe and Examining the Heads:**

In [5]:
teams_df = pd.read_csv('datasets/Teams.csv', delimiter=',')
teams_df.head()
#teams_df.shape

,Team_Id,Team_Name
0,1101,Abilene Chr
1,1102,Air Force
2,1103,Akron
3,1104,Alabama
4,1105,Alabama A&M


In [6]:
seasons_df = pd.read_csv('datasets/Seasons.csv', delimiter = ',')
seasons_df.head()
#seasons_df.shape

,Season,Dayzero,Regionw,Regionx,Regiony,Regionz
0,1985,10/29/1984,East,West,Midwest,Southeast
1,1986,10/28/1985,East,Midwest,Southeast,West
2,1987,10/27/1986,East,Southeast,Midwest,West
3,1988,11/2/1987,East,Midwest,Southeast,West
4,1989,10/31/1988,East,West,Midwest,Southeast


In [7]:
seasons_results_detailed_df = pd.read_csv('datasets/RegularSeasonDetailedResults.csv', delimiter = ',')
seasons_results_detailed_df.head()
#seasons_results_detailed_df.shape

#X = seasons_results_detailed_df.as_matrix(columns = ['Wscore'])
#y = seasons_results_detailed_df.as_matrix(columns = ['status'])
#y = np.ravel(y)

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [8]:
seasons_results_compact_df = pd.read_csv('datasets/RegularSeasonCompactResults.csv', delimiter = ',')
seasons_results_compact_df.head()
#seasons_results_compact_df.shape

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [9]:
tourney_results_detailed_df = pd.read_csv('datasets/TourneyDetailedResults.csv', delimiter = ',')
tourney_results_detailed_df.head()
#tourney_results_detailed_df.shape

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


In [10]:
tourney_results_compact_df = pd.read_csv('datasets/TourneyCompactResults.csv', delimiter = ',')
tourney_results_compact_df.head()
#tourney_results_compact_df.shape

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [11]:
tourney_seeds_df = pd.read_csv('datasets/TourneySeeds.csv', delimiter = ',')
tourney_seeds_df.head()
#tourney_seeds_df.shape


,Season,Seed,Team
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [12]:
tourney_slots_df = pd.read_csv('datasets/TourneySlots.csv', delimiter = ',')
tourney_slots_df.head()
#tourney_slots_df.shape

,Season,Slot,Strongseed,Weakseed
0,1985,R1W1,W01,W16
1,1985,R1W2,W02,W15
2,1985,R1W3,W03,W14
3,1985,R1W4,W04,W13
4,1985,R1W5,W05,W12


In [13]:
sample_df = pd.read_csv('datasets/SampleSubmission.csv', delimiter = ',')
sample_df.head()
#sample_df.shape

,Id,Pred
0,2016_1112_1114,0.5
1,2016_1112_1122,0.5
2,2016_1112_1124,0.5
3,2016_1112_1138,0.5
4,2016_1112_1139,0.5


In [14]:
seasons_results_detailed_df.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [15]:
tourney_results_detailed_df.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


In [14]:
def add_to_csv(df_lst, m = 'a'):
    #write pandas to csv
    count = 2003
    if m == 'a':
        for df in df_lst:
            #create csv and add pandas, use name
            df.to_csv('datasets/toruney_selected_reg_season_detailed_results/reg_season/%s_regular_seas.csv'%count)
            count += 1
    else:
        for df in df_lst:
            #create csv and add pandas, use name
            df.to_csv('datasets/toruney_selected_reg_season_detailed_results/tourney/%s_tourney.csv'%count)
            count += 1

In [17]:
#below is to make my training sets of regular season data

In [19]:
#create a dictionary to store which teams were in which years
yearly_dfs = []
teams_per_year = {}
for i in range(2003, 2016, 1):
    teams_per_year[i] = []

for key, value in teams_per_year.iteritems():
    year = tourney_results_detailed_df[tourney_results_detailed_df.Season == key]
    wteam_array = year['Wteam']
    team_array = np.append(wteam_array, year['Lteam'])
    teams_per_year[key] = sorted(np.unique(team_array))

for key, value in teams_per_year.iteritems():
    _yearly_df = pd.DataFrame(columns = list(seasons_results_detailed_df.columns.values))
    for index, row in seasons_results_detailed_df[seasons_results_detailed_df.Season == key].iterrows():
        if row['Wteam'] in value or row['Lteam'] in value:
            _yearly_df = _yearly_df.append(row)
    yearly_dfs.append(_yearly_df)

In [20]:
tourney_dfs = []
for i in range(2003, 2016, 1):
    tourney_dfs.append(tourney_results_detailed_df[tourney_results_detailed_df.Season == i])

In [23]:
add_to_csv(tourney_dfs, m = 'd')

In [24]:
#outputted wrong set
add_to_csv(yearly_dfs)

Preliminary Model Setup

In [24]:
### Assemble Season Long Stats
leagueData = {}

for index, game in seasons_results_detailed_df.iterrows():
    year = int(game['Season'])
    winningTeam = int(game['Wteam'])
    losingTeam = int(game['Lteam'])
    
    winningTeamStats = {}
    losingTeamStats = {}
    
    # we potentially want average and standard deviation of point differentials
    pointDifferential = int(game['Wscore']) - int(game['Lscore'])
    winningTeamStats['pointDifferential'] = pointDifferential
    losingTeamStats['pointDifferential'] = pointDifferential * -1
    
    winningTeamStats['fieldGoalsMade'] = int(game['Wfgm'])
    losingTeamStats['fieldGoalsMade'] = int(game['Lfgm'])
    
    winningTeamStats['fieldGoalsAtt'] = int(game['Wfga'])
    losingTeamStats['fieldGoalsAtt'] = int(game['Lfga'])
    
    winningTeamStats['freeThrowsMade'] = int(game['Wftm'])
    losingTeamStats['freeThrowsMade'] = int(game['Lftm'])
    
    winningTeamStats['freeThrowAtt'] = int(game['Wfta'])
    losingTeamStats['freeThrowAtt'] = int(game['Lfta'])
    
    winningTeamStats['offReb'] = int(game['Wor'])
    losingTeamStats['offReb'] = int(game['Lor'])
    
    winningTeamStats['assists'] = int(game['Wast'])
    losingTeamStats['assists'] = int(game['Last'])
    
    winningTeamStats['turnovers'] = int(game['Wto'])
    losingTeamStats['turnovers'] = int(game['Lto'])
    
    winningTeamStats['defReb'] = int(game['Wdr'])
    losingTeamStats['defReb'] = int(game['Ldr'])
    
    winningTeamStats['steals'] = int(game['Wstl'])
    losingTeamStats['steals'] = int(game['Lstl'])
    
    winningTeamStats['blocks'] = int(game['Wblk'])
    losingTeamStats['blocks'] = int(game['Lblk'])  

    winningTeamStats['fieldGoalsAllowed'] = int(game['Lfgm'])
    losingTeamStats['fieldGoalsAllowed'] = int(game['Wfgm'])
    
    winningTeamStats['oppFieldGoalsAtt'] = int(game['Lfga'])
    losingTeamStats['oppFieldGoalsAtt'] = int(game['Wfga'])
    
    winningTeamStats['freeThrowsAllowed'] = int(game['Lftm'])
    losingTeamStats['freeThrowsAllowed'] = int(game['Wftm'])
    
    winningTeamStats['oppFreeThrowsAtt'] = int(game['Lfta'])
    losingTeamStats['oppFreeThrowsAtt'] = int(game['Wfta'])
    
    categories = ['pointDifferential', 'fieldGoalsMade', 'fieldGoalsAtt', 'freeThrowsMade',
                  'freeThrowAtt', 'offReb', 'assists', 'turnovers', 'defReb', 'steals', 
                 'blocks', 'fieldGoalsAllowed', 'oppFieldGoalsAtt', 'freeThrowsAllowed', 
                 'oppFreeThrowsAtt']
    
    if year not in leagueData:
        leagueData[year] = {}

    if winningTeam in leagueData[year]:
        for category in categories:
            leagueData[year][winningTeam][category] += winningTeamStats[category]
        leagueData[year][winningTeam]['totalGames'] += 1
    else:
        leagueData[year][winningTeam] = winningTeamStats
        leagueData[year][winningTeam]['totalGames'] = 1
        
    if losingTeam in leagueData[year]:
        for category in categories:
            leagueData[year][losingTeam][category] += winningTeamStats[category]
        leagueData[year][losingTeam]['totalGames'] += 1
    else:
        leagueData[year][losingTeam] = losingTeamStats
        leagueData[year][losingTeam]['totalGames'] = 1

for year in leagueData:
    for team in leagueData[year]:
        totalTeamGames = leagueData[year][team]['totalGames']

        for category in leagueData[year][team]:
            leagueData[year][team][category] = leagueData[year][team][category]/float(totalTeamGames)
            
print len(leagueData)
print len(leagueData[2003])
print leagueData


14
327
{2016: {1101: {'pointDifferential': 9.222222222222221, 'fieldGoalsAtt': 53.629629629629626, 'freeThrowAtt': 24.62962962962963, 'blocks': 2.962962962962963, 'totalGames': 1.0, 'oppFieldGoalsAtt': 53.96296296296296, 'freeThrowsAllowed': 14.703703703703704, 'freeThrowsMade': 18.37037037037037, 'fieldGoalsMade': 26.37037037037037, 'fieldGoalsAllowed': 23.925925925925927, 'assists': 14.0, 'offReb': 8.37037037037037, 'defReb': 25.555555555555557, 'oppFreeThrowsAtt': 22.074074074074073, 'steals': 6.111111111111111, 'turnovers': 12.88888888888889}, 1102: {'pointDifferential': 10.7, 'fieldGoalsAtt': 55.2, 'freeThrowAtt': 23.9, 'blocks': 3.4, 'totalGames': 1.0, 'oppFieldGoalsAtt': 56.46666666666667, 'freeThrowsAllowed': 12.133333333333333, 'freeThrowsMade': 17.5, 'fieldGoalsMade': 25.1, 'fieldGoalsAllowed': 22.833333333333332, 'assists': 15.5, 'offReb': 9.466666666666667, 'defReb': 28.433333333333334, 'oppFreeThrowsAtt': 18.933333333333334, 'steals': 6.4, 'turnovers': 13.033333333333333},

In [29]:
trainingData = []
for index, game in tourney_results_detailed_df.iterrows():
    matchupVector = []
    year = int(game['Season'])
    
    possibleYears = [2003, 2004, 2005]
    
    if year in possibleYears:
        winningTeam = int(game['Wteam'])
        losingTeam = int(game['Lteam'])

        winningTeamStats = leagueData[year][winningTeam]
        losingTeamStats = leagueData[year][losingTeam]
        
        for category in categories:
            matchupVector.append(winningTeamStats[category] - losingTeamStats[category])
        
        trainingData.append(matchupVector + [1])
        matchupVector = list(map(lambda x: x*-1, matchupVector))
        trainingData.append(matchupVector + [0])
        
print trainingData

[[4.1, 2.3402298850574716, 5.083908045977012, 0.1999999999999993, -2.6896551724137936, 2.312643678160919, 1.4954022988505749, -2.724137931034482, -1.4701149425287348, 3.114942528735633, 1.168965517241379, -0.5678160919540254, -2.9919540229885087, 1.6402298850574724, -0.6275862068965523, 1], [-4.1, -2.3402298850574716, -5.083908045977012, -0.1999999999999993, 2.6896551724137936, -2.312643678160919, -1.4954022988505749, 2.724137931034482, 1.4701149425287348, -3.114942528735633, -1.168965517241379, 0.5678160919540254, 2.9919540229885087, -1.6402298850574724, 0.6275862068965523, 0], [4.754926108374384, 4.80665024630542, 9.264778325123153, 3.9618226600985214, 5.004926108374384, 2.336206896551724, 2.417487684729064, 1.7266009852216744, 1.6650246305418719, 0.7339901477832518, 0.2857142857142856, 4.285714285714285, 10.197044334975367, 1.3743842364532028, 2.3990147783251246, 1], [-4.754926108374384, -4.80665024630542, -9.264778325123153, -3.9618226600985214, -5.004926108374384, -2.3362068965517

In [30]:
trainingData = np.array(trainingData)
print trainingData

[[ 4.1         2.34022989  5.08390805 ...,  1.64022989 -0.62758621  1.        ]
 [-4.1        -2.34022989 -5.08390805 ..., -1.64022989  0.62758621  0.        ]
 [ 4.75492611  4.80665025  9.26477833 ...,  1.37438424  2.39901478  1.        ]
 ..., 
 [-1.89861751 -2.42626728 -6.07603687 ..., -0.52995392  0.47465438  0.        ]
 [ 2.24926686  2.20332356  3.89638319 ...,  3.17497556  3.64222874  1.        ]
 [-2.24926686 -2.20332356 -3.89638319 ..., -3.17497556 -3.64222874  0.        ]]


In [33]:
x_train = trainingData[:, :-1]
y_train = trainingData[:, -1]

logreg = linear_model.LogisticRegression()

# train logistic regression model using data from 2003-2005
logreg.fit(x_train, y_train)

# test trained model on 2006 data
testingData = []
for index, game in tourney_results_detailed_df.iterrows():
    matchupVector = []
    year = int(game['Season'])
    
    if year == 2006:
        winningTeam = int(game['Wteam'])
        losingTeam = int(game['Lteam'])

        winningTeamStats = leagueData[year][winningTeam]
        losingTeamStats = leagueData[year][losingTeam]
        
        for category in categories:
            matchupVector.append(winningTeamStats[category] - losingTeamStats[category])
        
        testingData.append(matchupVector + [1])
        matchupVector = list(map(lambda x: x*-1, matchupVector))
        testingData.append(matchupVector + [0])
        
testingData = np.array(testingData)
x_test = testingData[:, :-1]
y_test = testingData[:, -1]

print "For predicting the 2006 March Madness results, we get an accuracy of:", logreg.score(x_test, y_test)

For predicting the 2006 March Madness results, we get an accuracy of: 0.640625
